# BK tree test

In [11]:
import json
import random
import time
import matplotlib.pyplot as plt

## Import adminstrative region data

In [12]:
with open("./Data/region_tree.json", 'r') as file:
    region_tree = json.load(file)
    
with open("./Data/reverse_index_dist_to_prov.json", 'r') as file:
    rev_index_dist_to_prov = json.load(file)
    
with open("./Data/reverse_index_ward_to_prov.json", 'r') as file:
    rev_index_ward_to_prov = json.load(file)
    
with open("./Data/reverse_index_ward_to_dist.json", 'r') as file:
    rev_index_ward_to_dist = json.load(file)

In [13]:
all_provinces = [province['name'] for province in region_tree.values()]
all_districts = [district['name'] for district in rev_index_dist_to_prov.values()]
all_wards = [ward['name'] for ward in rev_index_ward_to_prov.values()]

## Levenshtein distance + Fuzzy matching

In [14]:
def levenshtein_dist(s1: str, s2: str) -> int:
    len_s1 = len(s1)
    len_s2 = len(s2)
    
    d = [[0]*(len_s2+1) for _ in range(len_s1+1)]
    
    for i in range(1, len_s1+1):
        d[i][0] = i
        
    for j in range(1, len_s2+1):
        d[0][j] = j
    
    for j in range(1, len_s2+1):
        for i in range(1, len_s1+1):
            if s1[i-1] == s2[j-1]:
                substitutionCost = 0
            else:
                substitutionCost = 1

            d[i][j] = min(d[i-1][j] + 1,
                          d[i][j-1] + 1,
                          d[i-1][j-1] + substitutionCost) 

    return d[-1][-1]

def approx_substring_matching_backtrace(e, f, best_match_pos: (int, int)) -> (int, int):
    i, j = best_match_pos
    while i != 0:
        #print(i, j)
        if f[i][j] == 0:
            i, j = i-1, j
        elif f[i][j] == 1:
            i, j = i, j-1
        elif f[i][j] == 2:
            i, j = i-1, j-1
        else:
            raise
    return (j, best_match_pos[1])

def approx_substring_matching(pattern: str, text: str) -> (int, (int, int)):
    len_p = len(pattern)
    len_t = len(text)
    
    e = [[0]*(len_t+1) for _ in range(len_p+1)]
    f = [[0]*(len_t+1) for _ in range(len_p+1)]

    for i in range(1, len_p+1):
        e[i][0] = i
        
    for j in range(1, len_t+1):
        f[0][j] = 1
    
    for j in range(1, len_t+1):
        for i in range(1, len_p+1):
            cost = [ e[i-1][j] + 1,
                     e[i][j-1] + 1,
                     e[i-1][j-1] + (pattern[i-1] != text[j-1]) ]
            
            f[i][j] = cost.index(min(cost))
            e[i][j] = min(cost)
    
    min_dist = min(e[len_p])    
    min_substr_end = e[len_p].index(min_dist)
    min_substr_end = min_substr_end - 1 if min_substr_end > 0 else min_substr_end
        
    return (min_dist, approx_substring_matching_backtrace(e, f, (len_p, min_substr_end)))

## BK-tree implementation

In [15]:
"""
https://github.com/benhoyt/pybktree
"""

from collections import deque
from operator import itemgetter

_getitem0 = itemgetter(0)


class BKTree(object):
    def __init__(self, distance_func, items=[], sorted_find=False):
        self.distance_func = distance_func
        self.tree = None
        self.sorted_find = sorted_find

        _add = self.add
        for item in items:
            _add(item)

    def add(self, item):
        node = self.tree
        if node is None:
            self.tree = (item, {})
            return

        # Slight speed optimization -- avoid lookups inside the loop
        _distance_func = self.distance_func

        while True:
            parent, children = node
            distance = _distance_func(item, parent)
            node = children.get(distance)
            if node is None:
                children[distance] = (item, {})
                break

    def find(self, item, n):
        if self.tree is None:
            return []

        candidates = deque([self.tree])
        found = []

        # Slight speed optimization -- avoid lookups inside the loop
        _candidates_popleft = candidates.popleft
        _candidates_extend = candidates.extend
        _found_append = found.append
        _distance_func = self.distance_func

        while candidates:
            candidate, children = _candidates_popleft()
            distance = _distance_func(candidate, item)
            if distance <= n:
                _found_append((distance, candidate))

            if children:
                lower = distance - n
                upper = distance + n
                _candidates_extend(c for d, c in children.items() if lower <= d <= upper)
        
        if self.sorted_find:
            found.sort(key=_getitem0)
        return found

    def __iter__(self):
        if self.tree is None:
            return

        candidates = deque([self.tree])

        # Slight speed optimization -- avoid lookups inside the loop
        _candidates_popleft = candidates.popleft
        _candidates_extend = candidates.extend

        while candidates:
            candidate, children = _candidates_popleft()
            yield candidate
            _candidates_extend(children.values())

In [16]:
bktree = BKTree(levenshtein_dist, ['help', 'hell', 'helps',
                                  'hello', 'shell', 'helper',
                                  'loop', 'troop'])

In [17]:
bktree.find('oop', 2)

[(1, 'loop'), (2, 'troop')]

## Build dictionary mapping word to region name

In [128]:
word_to_region = {}
word_count = 0
dupli_count = 0

for region_name in all_provinces:
    words = region_name.strip().split()
    
    for word in words:
        word_count += 1
        dupli_count += word in word_to_region
        if word in word_to_region:
            word_to_region[word]['province'].add(region_name)
        else:
            word_to_region[word] = {
                'province': set([region_name]),
                'district': set(),
                'ward': set()
            }

for region_name in all_districts:
    words = region_name.strip().split()
    
    for word in words:
        word_count += 1
        dupli_count += word in word_to_region
        if word in word_to_region:
            word_to_region[word]['district'].add(region_name)
        else:
            word_to_region[word] = {
                'province': set(),
                'district': set([region_name]),
                'ward': set()
            }
            
for region_name in all_wards:
    words = region_name.strip().split()
    
    for word in words:
        word_count += 1
        dupli_count += word in word_to_region
        if word in word_to_region:
            word_to_region[word]['ward'].add(region_name)
        else:
            word_to_region[word] = {
                'province': set(),
                'district': set(),
                'ward': set([region_name])
            }

In [129]:
print(word_count, dupli_count)
print(len(word_to_region))
print(word_to_region)

16611 14577
2034
{'Hà': {'province': {'Hà Nội', 'Hà Tĩnh', 'Hà Nam', 'Hà Giang'}, 'district': {'Thạch Hà', 'Hà Tiên', 'Lộc Hà', 'Hà Đông', 'Thanh Hà', 'Hà Tĩnh', 'Đầm Hà', 'Sơn Hà', 'Hưng Hà', 'Hà Quảng', 'Lâm Hà', 'Hải Hà', 'Đắk Hà', 'Bắc Hà', 'Hà Trung', 'Hà Giang', 'Đông Hà'}, 'ward': {'Hà Lĩnh', 'Hồng Hà', 'Đại Hà', 'Sóc Hà', 'Đông Hà', 'Quảng Hà', 'Hà Mòn', 'Xuân Hà', 'Hưng Hà', 'Hà Bình', 'Hà Thái', 'Hà Huy Tập', 'Hà Lâm', 'Hà Tam', 'Hà Cầu', 'Thuận Hà', 'Hà Lộc', 'Tân Hà', 'Long Hà', 'Hà Giang', 'Khuôn Hà', 'Thạch Hà', 'Hà Tây', 'Trung Hà', 'Đăng Hà', 'Hà Phong', 'Hà Vinh', 'Phước Hà', 'Duyên Hà', 'Hà Đông', 'Mỹ Hà', 'Nam Hà', 'Phủ Hà', 'Đắk Hà', 'Lộc Hà', 'Dương Hà', 'Hoằng Hà', 'Vân Hà', 'Hà Thạch', 'Hà Lâu', 'Hà Bầu', 'An Hà', 'Hà Kỳ', 'Thanh Hà', 'Phúc Hà', 'Quân Hà', 'Bắc Hà', 'Trường Hà', 'Hà Tiến', 'Liên Hà', 'Hà Châu', 'La Hà', 'Hà Ngọc', 'Tiên Hà', 'Nghĩa Hà', 'Hà Hồi', 'Hà Long', 'Kỳ Hà', 'Hà Lang', 'Hà Khánh', 'Hà Lai', 'Ngọc Hà', 'Hà Lầm', 'Hà Tân', 'Vinh Hà', 'Giao 

## Build BK-tree of region names

In [130]:
bktree_region_word = BKTree(levenshtein_dist, word_to_region.keys())

In [131]:
bktree_region_word.find('Tkng', 1)

[(1, 'Tằng'),
 (1, 'Tang'),
 (1, 'Tăng'),
 (1, 'Tòng'),
 (1, 'Tung'),
 (1, 'Tủng'),
 (1, 'Tông'),
 (1, 'Tùng'),
 (1, 'Túng'),
 (1, 'Tổng'),
 (1, 'Tụng'),
 (1, 'Tống'),
 (1, 'Tầng'),
 (1, 'Ting')]

In [132]:
start = time.perf_counter_ns()
for _ in range(100):
    bktree_region_word.find('Tkng', 1)
runtime = time.perf_counter_ns() - start

print(runtime / 1_000_000, 'ms')

356.800462 ms


## Find region with BK-tree

In [133]:
addrs = ['Ea. Knốp', 'AnE Minh', 'Phú Lươnz ']

def test_find_region_with_bktree(addr):
    words = [w.strip() for w in addr.split()]
    prov_candidate = dist_candidate = ward_candidate = None

    for word in words:
        suggestions = [sug[1] for sug in bktree_region_word.find(word, 1)]
        possible_prov = possible_dist = possible_ward = set()

        for sug in suggestions:
            possible_prov = possible_prov | word_to_region[sug]['province']
            possible_dist = possible_dist | word_to_region[sug]['district']
            possible_ward = possible_ward | word_to_region[sug]['ward']

        if prov_candidate is None:
            prov_candidate = possible_prov
        else:
            prov_candidate = prov_candidate & possible_prov

        if dist_candidate is None:
            dist_candidate = possible_dist
        else:
            dist_candidate = dist_candidate & possible_dist

        if ward_candidate is None:
            ward_candidate = possible_ward
        else:
            ward_candidate = ward_candidate & possible_ward

        #print('word:', word)
        #print('sugg:', suggestions)
        #print('prov: {}\ndist: {}\nward: {}'.format(possible_prov, possible_dist, possible_ward))
        #print('-' * 5)

    #print('prov: {}\ndist: {}\nward: {}'.format(prov_candidate, dist_candidate, ward_candidate))
    #print('-' * 5)
        
    return (prov_candidate, dist_candidate, ward_candidate)

In [138]:
start = time.perf_counter_ns()
for _ in range(100):
    choice = random.randrange(0, len(addrs))
    result = test_find_region_with_bktree(addrs[choice])
runtime = time.perf_counter_ns() - start

print(runtime / 1_000_000, 'ms')

437.549803 ms


In [140]:
test_find_region_with_bktree(addrs[2])

(set(),
 {'Phú Lương'},
 {'Lương Phi', 'Lương Phú', 'Phù Lương', 'Phú Lương', 'Phúc Lương'})